## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

# Error Logs

> - Set up weekly cycle error log job (Q1)
> - Read all the event logs (Q2)
> - (Q3)
> - (Q4)
> - (Q5)

> - Set up weekly cycle error log job (Q1)

In [ ]:
/* Set up weekly cycle error log job 8AM Sunday */
USE [msdb]
GO

/****** Object:  Job [DBA - Weekly Error Log Recycle]    Script Date: 23/12/2021 12:26:19 ******/
BEGIN TRANSACTION
DECLARE @ReturnCode INT
SELECT @ReturnCode = 0
/****** Object:  JobCategory [[Uncategorized (Local)]]    Script Date: 23/12/2021 12:26:19 ******/
IF NOT EXISTS (SELECT name FROM msdb.dbo.syscategories WHERE name=N'[Uncategorized (Local)]' AND category_class=1)
BEGIN
EXEC @ReturnCode = msdb.dbo.sp_add_category @class=N'JOB', @type=N'LOCAL', @name=N'[Uncategorized (Local)]'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback

END

DECLARE @jobId BINARY(16)
EXEC @ReturnCode =  msdb.dbo.sp_add_job @job_name=N'DBA - Weekly Error Log Recycle', 
		@enabled=1, 
		@notify_level_eventlog=0, 
		@notify_level_email=0, 
		@notify_level_netsend=0, 
		@notify_level_page=0, 
		@delete_level=0, 
		@description=N'No description available.', 
		@category_name=N'[Uncategorized (Local)]', 
		@owner_login_name=N'sa', @job_id = @jobId OUTPUT
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
/****** Object:  Step [Recycle errorlog]    Script Date: 23/12/2021 12:26:19 ******/
EXEC @ReturnCode = msdb.dbo.sp_add_jobstep @job_id=@jobId, @step_name=N'Recycle errorlog', 
		@step_id=1, 
		@cmdexec_success_code=0, 
		@on_success_action=1, 
		@on_success_step_id=0, 
		@on_fail_action=2, 
		@on_fail_step_id=0, 
		@retry_attempts=0, 
		@retry_interval=0, 
		@os_run_priority=0, @subsystem=N'TSQL', 
		@command=N'EXEC sp_cycle_errorlog', 
		@database_name=N'master', 
		@flags=0
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_update_job @job_id = @jobId, @start_step_id = 1
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_add_jobschedule @job_id=@jobId, @name=N'Weekly 8AM Sunday', 
		@enabled=1, 
		@freq_type=8, 
		@freq_interval=1, 
		@freq_subday_type=1, 
		@freq_subday_interval=0, 
		@freq_relative_interval=0, 
		@freq_recurrence_factor=1, 
		@active_start_date=20210920, 
		@active_end_date=99991231, 
		@active_start_time=80000, 
		@active_end_time=235959, 
		@schedule_uid=N'7319c221-e0d9-454c-9e41-07ce47650768'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_add_jobserver @job_id = @jobId, @server_name = N'(local)'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
COMMIT TRANSACTION
GOTO EndSave
QuitWithRollback:
    IF (@@TRANCOUNT > 0) ROLLBACK TRANSACTION
EndSave:
GO




> - Read all the event logs (Q2)

In [3]:
/* Read Error logs.sql  #SQL */
--https://thelonedba.wordpress.com/2019/08/16/reading-sql-server-error-logs/

SET NOCOUNT ON
DECLARE @daysback INT = 0; -- number of days to go back in the logs.  0 = today only
 
-- table variable for holding the details of the error logs.
-- Yes, I know, table variables are evil.  This one is unlikely to hold more than a few dozen rather narrow lines
DECLARE @ErrorLogs TABLE
(
    Archive INT NOT NULL,
    LogDate DATETIME NOT NULL,
    LogFileSizeBytes BIGINT NOT NULL,
    ReadThis TINYINT NULL
);
 
-- useful trick if you don't know it: INSERT INTO a table the results of EXECing a SP.
INSERT INTO @ErrorLogs (Archive, LogDate, LogFileSizeBytes)
EXEC sys.sp_enumerrorlogs;
 
DECLARE @lognum INT = 0;
DECLARE @logdate DATETIME;
 
--figure out which logfiles we need.
WITH
    NextLog
  AS
  (
      SELECT    Archive,
                LogDate,
                LogFileSizeBytes,
                ReadThis,
                ISNULL(LAG(LogDate) OVER (ORDER BY Archive), LogDate) AS nextlogdate
      FROM      @ErrorLogs
  )
UPDATE          @ErrorLogs
SET             ReadThis = CASE
                               WHEN e.Archive = 1 THEN 1 -- always read the first file; doesn't always get identified by the next line
                               WHEN n.nextlogdate >= DATEADD(DAY, DATEDIFF(DAY, '20100101', GETDATE()) - ABS(@daysback), '20100101') THEN
                                   1
                               ELSE
                                   0
                           END
FROM            NextLog    AS n
    INNER JOIN  @ErrorLogs AS e ON e.Archive = n.Archive;
 
--just checking which files we're looking at...  Probably comment this line out for production use
SELECT      *
FROM        @ErrorLogs
ORDER BY    Archive;
 
IF OBJECT_ID('tempdb.dbo.#spErrorLog', 'U') IS NOT NULL BEGIN
    DROP TABLE #spErrorLog;
END;
 
CREATE TABLE #spErrorLog (logdate DATETIME NOT NULL, ProcessInfo VARCHAR(20) NULL, Text VARCHAR(MAX) NOT NULL);
 
WHILE @lognum <= (SELECT    MAX(Archive)FROM   @ErrorLogs WHERE ReadThis = 1)
BEGIN
    INSERT INTO #spErrorLog
    EXEC sys.sp_readerrorlog @p1 = @lognum;
 
    SELECT  @lognum = @lognum + 1;
 
    SELECT  @logdate = LogDate
    FROM    @ErrorLogs
    WHERE   Archive = @lognum;
END;
 
SELECT      *
FROM        #spErrorLog
WHERE
            1 = 1
  AND
            (
                ProcessInfo = 'Server' -- we like server-related boot messages.
           OR
                ( -- filter out noisy messages that we don't really need to see here
                    Text NOT LIKE '%Backup Log%'
              AND   Text NOT LIKE 'Log was backed up%'
              AND   Text NOT LIKE 'CHECKDB for database % finished without errors on %This is an informational message only; no user action is required.'
              AND   Text NOT LIKE 'DBCC CHECKDB % WITH all_errormsgs, no_infomsgs, data_purity executed by % found 0 errors and repaired 0 errors.%'
              AND   Text NOT LIKE 'BACKUP DATABASE WITH DIFFERENTIAL successfully processed % pages%'
              AND   Text NOT LIKE 'Database differential changes were backed up. Database: % This is an informational message. No user action is required.'
              AND   Text NOT LIKE 'BACKUP DATABASE successfully processed % pages %'
              AND   Text NOT LIKE 'Database backed up. Database: % This is an informational message only. No user action is required.'
              AND   logdate >= DATEADD(DAY, DATEDIFF(DAY, '20100101', GETDATE()) - ABS(@daysback), '20100101')
                )
            )
ORDER BY    logdate;


Commands completed successfully.

Total execution time: 00:00:00.108

Archive,LogDate,LogFileSizeBytes,ReadThis
0,2022-02-21 17:20:00.000,30910,1
1,2022-02-19 10:55:00.000,22546,1
2,2022-02-18 17:18:00.000,28502,0
3,2022-02-17 17:14:00.000,27140,0
4,2022-02-16 19:54:00.000,24886,0
5,2022-02-16 17:20:00.000,33220,0
6,2022-02-15 15:26:00.000,39112,0


logdate,ProcessInfo,Text
2022-02-19 10:49:51.050,Server,Microsoft SQL Server 2019 (RTM-GDR) (KB4583458) - 15.0.2080.9 (X64) Nov 6 2020 16:50:01 Copyright (C) 2019 Microsoft Corporation Developer Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 22000: )
2022-02-19 10:49:51.080,Server,UTC adjustment: 0:00
2022-02-19 10:49:51.080,Server,(c) Microsoft Corporation.
2022-02-19 10:49:51.080,Server,All rights reserved.
2022-02-19 10:49:51.080,Server,Server process ID is 6432.
2022-02-19 10:49:51.080,Server,"System Manufacturer: 'ASUSTeK COMPUTER INC.', System Model: 'ZenBook Pro 15 UX550GDX_UX580GD'."
2022-02-19 10:49:51.080,Server,Authentication mode is WINDOWS-ONLY.
2022-02-19 10:49:51.080,Server,Logging SQL Server messages in file 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\ERRORLOG'.
2022-02-19 10:49:51.080,Server,The service account is 'NT Service\MSSQLSERVER'. This is an informational message; no user action is required.
2022-02-19 10:49:51.080,Server,Registry startup parameters: -d C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\master.mdf -e C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\ERRORLOG -l C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\mastlog.ldf


## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)